# ТЕМА 2. 

Событийно-ориентированное программирование.

## Лекция. 
Работа с многопоточностью и многопроцессорностью в Qt

### Учебные вопросы

1. QRunnable + QThreadPool
2. QProcess

## Источники

* Официальная документация: https://doc.qt.io/qtforpython/tutorials

* Прохоренок Н. А., Дронов В. А. Python 3 и PyQt 5. Разработка приложений. 2019 г. 

# 1. QRunnable + QThreadPool

**QRunnable + QThreadPool**

Qt предоставляет очень простой интерфейс для выполнения заданий в других потоках. 

Использование этой возможности построено вокруг двух классов: `QRunnable` и `QThreadPool`. 

**`QRunnable`**- это контейнер для работы, которую вы хотите выполнить (представляет задачу или фрагмент кода, который необходимо запустить в потоке).

**`QThreadPool`** - это менеджер контейнеров, с помощью которого вы передаете работу помещенную в контейнер альтернативным потокам.

Каждое приложение имеет глобальный пул потоков. Получить ссылку на него можно вызвав `QThreadPool.globalInstance()`.

Глобальный пул потоков поддерживает и управляет рекомендуемым количеством потоков, обычно основанным на количестве ядер в вашем текущем процессоре. Он также обрабатывает постановку в очередь и выполнение задач в потоках вашего приложения. Потоки в пуле можно использовать повторно, что предотвращает накладные расходы, связанные с созданием и уничтожением потоков.

Чтобы создавать задачи и запускать их в пуле потоков, используется класс `QRunnable`.

Плюсом использования `QThreadPool` является удобство, необходимо только ставить задачи в очередь и получать результаты.

Для определения собственного класса `QRunnable` необходимо:

1. Создать свой класс унаследовавшись от `QRunnable`.
2. Переопределить метод run(), поместив в него код, который необходимо выполнить.
3. Создать экземпляр своего класса в базоваом классе и запустить его.
4. Создать экземпляр класса QThreadPool.
5. Запустить их.

```python
"""
Использование классов QRunnable и QThreadPool для создания сразу множества задач
"""

import logging
import random
import time

from PySide6 import QtCore, QtWidgets

logging.basicConfig(format="%(message)s", level=logging.INFO)


class Runnable(QtCore.QRunnable):
    def __init__(self, thread_number):
        super().__init__()
        self.thread_number = thread_number

    def run(self) -> None:
        """
        Имитация выполнения долгой функции

        :return: None
        """

        for i in range(5):
            logging.info(f"Working in thread {self.thread_number}, step {i + 1}/5")
            time.sleep(random.randint(700, 2500) / 1000)


class Window(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.initUi()
        self.initSignals()

    def initUi(self) -> None:
        """
        Инициализация Ui

        :return: None
        """

        self.resize(250, 150)
        self.setWindowTitle("QThreadPool + QRunnable")

        self.label = QtWidgets.QLabel("Hello!")
        self.label.setAlignment(QtCore.Qt.AlignmentFlag.AlignCenter)

        self.pushButton = QtWidgets.QPushButton("Выполнить задачи")

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.pushButton)

        self.setLayout(layout)

    def initSignals(self) -> None:
        """
        Инициализация сигналов

        :return: None
        """

        self.pushButton.clicked.connect(self.runTasks)

    def runTasks(self) -> None:
        """
        Запуск всех задач в пуле потоков

        :return: None
        """

        threadCount = QtCore.QThreadPool.globalInstance().maxThreadCount()
        self.label.setText(f"Running {threadCount} Threads")

        threadPool = QtCore.QThreadPool.globalInstance()
        for thread_number in range(threadCount):
            runnable = Runnable(thread_number)

            threadPool.start(runnable)


if __name__ == '__main__':
    app = QtWidgets.QApplication()

    window = Window()
    window.show()

    app.exec()

```

**QThread** vs **QRunnable + QThreadPool**

Если в приложении используется очень много потоков, то можно столкнуться со значительными накладными расходами, связанными с созданием и уничтожением потоков. 

Вам также нужно учитывать, сколько потоков вы можете запустить в данной системе, чтобы ваше приложение оставалось эффективным. 

**QRunnable** хорошо подходит для ситуаций, когда необходимо выполнить некоторую фоновую обработку в одном или нескольких вторичных потоках, не требуя полной мощности и гибкости, предоставляемых QThread.

> Задачи, для которых не требуется цикл событий. В частности, задачи, которые не используют механизм сигналов / слотов во время выполнения задачи. Используйте QThreadPool + QRunnable.

**QThread** подходит для ситуаций, когда необходимо обмениваться данными между потоком и приложением, т.к. в классе уже реализована поддержка сеханизма сигналов/слотов, для управления потоком.

> Задачи, которые используют сигналы / слоты и, следовательно, нуждаются в цикле событий. Используйте QThread.

# 2. QProcess

Мы рассмотрели, как выполнять работу в отдельных потоках, это позволяет выполнять сложные задачи, не "замораживая" пользовательский интерфейс. 

Это отлично работает при использовании библиотек Python для выполнения задач, но иногда требуется запускать внешние приложения, передавая параметры и получая результаты.

Для таких задач можно воспользоваться модулем QProcess.

```python
"""
Простейшее использование QProcess
"""

from PySide6 import QtWidgets, QtCore


class Window(QtWidgets.QWidget):

    def __init__(self, parent=None):
        super().__init__(parent)

        self.process = None

        self.initUi()
        self.initSignals()

    def initUi(self) -> None:
        """
        Инициализация Ui

        :return: None
        """

        self.pushButton = QtWidgets.QPushButton("Показать список файлов")

        self.plainTextEdit = QtWidgets.QPlainTextEdit()
        self.plainTextEdit.setReadOnly(True)

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.pushButton)
        layout.addWidget(self.plainTextEdit)

        self.setLayout(layout)

    def initSignals(self) -> None:
        """
        Инициализация сигналов

        :return: None
        """

        self.pushButton.clicked.connect(self.executeOtherProcess)

    def executeOtherProcess(self) -> None:
        """
        Запуск выполнения другого процесса

        :return: None
        """

        if self.process is None:
            self.plainTextEdit.appendPlainText("Запуск другого процесса")
            self.process = QtCore.QProcess()
            self.process.start("python", ['c_other_py_script.py'])
            self.process.finished.connect(self.processFinished)

    def processFinished(self) -> None:
        """
        Действие при завершении другого процесса

        :return: None
        """

        self.plainTextEdit.appendPlainText("Другой процесс завершен")
        self.process = None


if __name__ == '__main__':
    app = QtWidgets.QApplication()

    window = Window()
    window.show()

    app.exec()
```

`QProcess` предоставляет ряд сигналов, которые могут быть использованы для отслеживания хода и состояния процессов.

> Механизм очень похож на работу с модулем `subprocess`  стандартной библиотеки.

Существует возможность обращения к потокам ввода/вывода, которые используются для извлечения данных из запущенного процесса. 

Два стандартных потока - это стандартный вывод и стандартная ошибка. Первый получает результирующие данные, которые выводит приложение, в то время как второй получает диагностические сообщения или сообщения об ошибках.

В Qt используются те же принципы. `QProcess` объект имеет два сигнала `.readyReadStandardOutput`, `.readyReadStandardError` которые используются для уведомления о наличии данных в соответствующих потоках. 

```python
"""
Простейшее использование QProcess с получением данных
"""

from PySide6 import QtWidgets, QtCore


class Window(QtWidgets.QWidget):

    def __init__(self, parent=None):
        super().__init__(parent)

        self.process = None

        self.initUi()
        self.initSignals()

    def initUi(self) -> None:
        """
        Инициализация Ui

        :return: None
        """

        self.pushButton = QtWidgets.QPushButton("Показать список файлов")

        self.plainTextEdit = QtWidgets.QPlainTextEdit()
        self.plainTextEdit.setReadOnly(True)

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.pushButton)
        layout.addWidget(self.plainTextEdit)

        self.setLayout(layout)

    def initSignals(self) -> None:
        """
        Инициализация сигналов

        :return: None
        """

        self.pushButton.clicked.connect(self.executeOtherProcess)

    def executeOtherProcess(self) -> None:
        """
        Запуск выполнения другого процесса

        :return: None
        """

        if self.process is None:
            self.plainTextEdit.appendPlainText("Выполнение процесса")
            self.process = QtCore.QProcess()
            self.process.readyReadStandardOutput.connect(self.handleOutput)
            self.process.readyReadStandardError.connect(self.handleError)
            self.process.stateChanged.connect(self.handleStateChange)
            self.process.finished.connect(self.processFinished)
            # self.process.start("python", ["c_other_py_script.py"])  # запуск py скрипта в отдельном потоке
            self.process.start("ping", ["8.8.8.8"])  # запуск команды ping в отдельном потоке

    def handleError(self) -> None:
        """
        Обработка данных из потока stderr

        :return: None
        """

        data = self.process.readAllStandardError()
        stderr = bytes(data).decode("utf8")
        self.plainTextEdit.appendPlainText(stderr)

    def handleOutput(self) -> None:
        """
        Обработка данных из потока stdout

        :return: None
        """

        data = self.process.readAllStandardOutput()
        stdout = bytes(data).decode("utf8")
        self.plainTextEdit.appendPlainText(stdout)

    def handleStateChange(self, state) -> None:
        """
        Изменение статуса потока

        :param state: статус
        :return: None
        """

        states = {
            QtCore.QProcess.NotRunning: 'Not running',
            QtCore.QProcess.Starting: 'Starting',
            QtCore.QProcess.Running: 'Running',
        }
        state_name = states[state]
        self.plainTextEdit.appendPlainText(f"Состояние изменено: {state_name}")

    def processFinished(self) -> None:
        """
        Обработка завершения потока

        :return: None
        """

        self.plainTextEdit.appendPlainText("Процесс завершен")
        self.process = None


if __name__ == '__main__':
    app = QtWidgets.QApplication()

    window = Window()
    window.show()

    app.exec()
```

# Итоги

Лучшие практики использования многопоточности в Qt, которые вы можете применить в своём приложении:

* Избегайте запуска длительных задач в главном потоке приложения PyQt.

* Используйте `QThread` объекты и для создания рабочих потоков.

* Используйте `QThreadPool` и `QRunnable`, если вам нужно управлять пулом рабочих потоков.

* Используйте сигналы и слоты для установления безопасной межпотоковой связи.

* Не пытайтесь создавать, получать доступ или обновлять компоненты или виджеты GUI из рабочего потока.

Если Вы грамотно применяете эти рекомендации при работе с потоками в Qt, то ваши приложения будут менее подвержены ошибкам и будут более стойкими и надежными. Вы предотвратите такие проблемы, как повреждение данных, взаимоблокировки, условия гонки и другие.


**Заключение**

Выполнение длительных задач в главном потоке приложения PyQt может привести к зависанию графического интерфейса приложения и его отказу отвечать на запросы. 

Это распространенная проблема в программировании приложений с графическим интерфейсом и может привести к плохому взаимодействию с пользователем.

Создание рабочих потоков с помощью `QThread` для обработки длительных задач эффективно решает эту проблему в ваших приложениях с графическим интерфейсом.